In [10]:
import os
import subprocess
from IPython.display import IFrame

## Precursors

In [2]:
if not os.path.isfile('data/hg19.ml.fa'):
    subprocess.call('curl -o data/hg19.ml.fa https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa', shell=True)
    subprocess.call('curl -o data/hg19.ml.fa.fai https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa.fai', shell=True)                

In [3]:
if not os.path.isdir('models/heart'):
    os.mkdir('models/heart')
if not os.path.isfile('models/heart/model_best.h5'):
    subprocess.call('curl -o models/heart/model_best.h5 https://storage.googleapis.com/basenji_tutorial_data/model_best.h5', shell=True)

In [4]:
lines = [['index','identifier','file','clip','sum_stat','description']]
lines.append(['0', 'CNhs11760', 'data/CNhs11760.bw', '384', 'sum', 'aorta'])
lines.append(['1', 'CNhs12843', 'data/CNhs12843.bw', '384', 'sum', 'artery'])
lines.append(['2', 'CNhs12856', 'data/CNhs12856.bw', '384', 'sum', 'pulmonic_valve'])

samples_out = open('data/heart_wigs.txt', 'w')
for line in lines:
    print('\t'.join(line), file=samples_out)
samples_out.close()

## Compute scores

Saturation mutagenesis is a powerful tool both for dissecting a specific sequence of interest and understanding what the model learned. [basenji_sat_bed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sat_bed.py) enables this analysis from a test set of data. [basenji_sat_vcf.py](https://github.com/calico/basenji/blob/master/bin/basenji_sat_vcf.py) lets you provide a VCF file for variant-centered mutagenesis.

To do this, you'll need
 * Trained model
 * BED file

First, you can either train your own model in the [Train/test tutorial](https://github.com/calico/basenji/blob/master/tutorials/train_test.ipynb) or use one that I pre-trained from the models subdirectory.

We'll bash the GATA4 promoter to see what motifs drive its expression. I placed a BED file surrounding the GATA4 TSS in data/gata4.bed, so we'll use [basenji_sat_bed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sat_bed.py).

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -f | data/hg19.ml.fa | Genome FASTA to extract sequences. |
| -l | 200 | Saturation mutagenesis region in the center of the given sequence(s) |
| -o | gata4_sat | Outplot plot directory. |
| --rc | True | Predict forward and reverse complement versions and average the results. |
| -t | data/heart_wigs.txt | Target indexes to analyze. |
| params_file | models/params_small.json | JSON specified parameters to setup the model architecture and optimization parameters. |
| model_file | models/heart/model_best.h5 | Trained saved model parameters. |
| input_file | data/gata4.bed | BED regions. |

In [6]:
! basenji_sat_bed.py -f data/hg19.ml.fa -l 200 -o output/gata4_sat --rc -t data/heart_wigs.txt models/params_small.json models/heart/model_best.h5 data/gata4.bed

2021-02-12 15:54:41.261696: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-12 15:54:41.261834: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence (InputLayer)           [(None, 131072, 4)]  0                                            
__________________________________________________________________________________________________
stochastic_reverse_complement ( ((None, 131072, 4),  0           sequence[0][0]                   
__________

## Plot

The saturation mutagenesis scores go into output/gata4_sat/scores.h5. Then we can use [basenji_sat_plot.py](https://github.com/calico/basenji/blob/master/bin/basenji_sat_plot.py) to visualize the scores.

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -g | True | Draw a sequence logo for the gain score, too, identifying repressor motifs. |
| -l | 200 | Saturation mutagenesis region in the center of the given sequence(s) |
| -o | output/gata4_sat/plots | Outplot plot directory. |
| -t | data/heart_wigs.txt | Target indexes to analyze. |
| scores_file | output/gata4_sat/scores.h5 | Scores HDF5 from above. |

In [7]:
! basenji_sat_plot.py --png -l 200 -o output/gata4_sat/plots -t data/heart_wigs.txt output/gata4_sat/scores.h5

/Users/drk/code/Basenji/bin/basenji_sat_plot.py:98: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  scores_h5 = h5py.File(scores_h5_file)
/Users/drk/code/Basenji/bin/basenji_sat_plot.py:194: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  plt.tight_layout()


In [8]:
! ls output/gata4_sat/plots

seq0_t0.png seq0_t1.png seq0_t2.png


The resulting plots reveal a low level of activity, with a GC-rich motif driving the only signal.

In [19]:
IFrame('output/gata4_sat/plots/seq0_t0.png', width=1200, height=400)